In [36]:
from buildingmotif import BuildingMOTIF
bm = BuildingMOTIF("sqlite://") # in-memory instance

In [37]:
from rdflib import Namespace
from buildingmotif.dataclasses import Model

# create the namespace
BLDG = Namespace('test:building/')

# Create the model! This will raise an exception if the namespace is not syntactically valid.
model = Model.create(BLDG, description="This is a test model for a simple building") 

In [38]:
print(model.graph.serialize())

@prefix owl: <http://www.w3.org/2002/07/owl#> .

<test:building/> a owl:Ontology .




# Loading Libraries

In [39]:
# load a library
from buildingmotif.dataclasses import Library
brick = Library.load(ontology_graph="Brick-full.ttl")

# print the first 10 templates
print("The Brick library contains the following templates:")
for template in brick.get_templates()[:10]:
    print(f"  - {template.name}")
    

2023-01-13 11:44:12,996 | root |  WARNING: Warning: could not find dependee http://data.ashrae.org/bacnet/2020#BACnetDevice
2023-01-13 11:44:12,997 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject


The Brick library contains the following templates:
  - https://brickschema.org/schema/Brick#Intercom_Equipment
  - https://brickschema.org/schema/Brick#Static_Pressure_Deadband_Setpoint
  - https://brickschema.org/schema/Brick#Warm_Cool_Adjust_Sensor
  - https://brickschema.org/schema/Brick#Max_Load_Setpoint
  - https://brickschema.org/schema/Brick#Manual_Auto_Status
  - https://brickschema.org/schema/Brick#Enable_Differential_Enthalpy_Command
  - https://brickschema.org/schema/Brick#Frost_Sensor
  - https://brickschema.org/schema/Brick#Return_Fan
  - https://brickschema.org/schema/Brick#IlluminanceShape
  - https://brickschema.org/schema/Brick#Radon_ConcentrationShape


# Exploring a Template

In [40]:
# import this to make writing URIs easier
from buildingmotif.namespaces import BRICK


"""
# get template
ahu_template = brick.get_template_by_name(BRICK.AHU)



print(ahu_template)
# print template parameters

print("The template has the following parameters:")
for param in ahu_template.parameters:
    print(f"  {param}")
"""

'\n# get template\nahu_template = brick.get_template_by_name(BRICK.AHU)\n\n\n\nprint(ahu_template)\n# print template parameters\n\nprint("The template has the following parameters:")\nfor param in ahu_template.parameters:\n    print(f"  {param}")\n'

In [41]:
#print(ahu_template.body.serialize())


# Evaluating a Template

In [42]:
"""
ahu_name = "AHU_AC"
ahu_binding_dict = {"name": BLDG[ahu_name]}
ahu_graph = ahu_template.evaluate(ahu_binding_dict)

# ahu_graph is just an instance of rdflib.Graph
print(ahu_graph.serialize())
"""

'\nahu_name = "AHU_AC"\nahu_binding_dict = {"name": BLDG[ahu_name]}\nahu_graph = ahu_template.evaluate(ahu_binding_dict)\n\n# ahu_graph is just an instance of rdflib.Graph\nprint(ahu_graph.serialize())\n'

# Adding Evaluated Templates to the Model

In [43]:
"""
model.add_graph(ahu_graph)
print(model.graph.serialize())
"""

'\nmodel.add_graph(ahu_graph)\nprint(model.graph.serialize())\n'

In [44]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [45]:
df=pd.read_csv('entity_relations_v3.csv')
print(df)

   subjectURI     brick_class relation   objectURI   brick_class.1
0        FCU1             FCU    feeds       Zone1       HVAC_Zone
1        FCU2             FCU    feeds       Zone1       HVAC_Zone
2        FCU3             FCU    feeds       Zone1       HVAC_Zone
3        FCU4             FCU    feeds       Zone1       HVAC_Zone
4        FCU5             FCU    feeds       Zone1       HVAC_Zone
5    Furnace1  HVAC_Equipment    feeds       Zone1       HVAC_Zone
6    Furnace2  HVAC_Equipment    feeds       Zone1       HVAC_Zone
7    Furnace1  HVAC_Equipment  hasPart     Boiler1          Boiler
8    Furnace1  HVAC_Equipment  hasPart        Fan1             Fan
9    Furnace1  HVAC_Equipment  hasPart     HeatEx1  Heat_Exchanger
10   Furnace2  HVAC_Equipment  hasPart     Boiler2          Boiler
11   Furnace2  HVAC_Equipment  hasPart        Fan2             Fan
12   Furnace2  HVAC_Equipment  hasPart     HeatEx2  Heat_Exchanger
13    Boiler1          Boiler    feeds     HeatEx1  Heat_Excha

In [46]:
import re
def uni_encode(mystring):
    mystring=str(mystring)
    mystring=re.sub(r'[^\w\s]', '', mystring)
    mynewstring=mystring.replace(" ", "_")
    return mynewstring
df=df.applymap(lambda x: uni_encode(x) if pd.notnull(x) else x)

In [47]:

#print(equipment_template)

In [48]:
length_dataframe=len(df)
#print(length_dataframe)

for row_df in range(0,length_dataframe):
    for column_df in [0,3]:
        brick_class=df.iloc[row_df,column_df+1]
        dummy_code = f"equipment_template= brick.get_template_by_name(BRICK.{brick_class})"
        exec(dummy_code)
        equipment_name =df.iloc[row_df,column_df]
        equipment_binding_dict = {"name": BLDG[equipment_name]}
        equipment_graph = equipment_template.evaluate(equipment_binding_dict)
        model.add_graph(equipment_graph)




In [49]:
for row_df in range(0,length_dataframe):
    equipment_name1 =df.iloc[row_df,0]
    equipment_name2 =df.iloc[row_df,3]
    BRICK_relation=df.iloc[row_df,2]
    dummy_code=f"model.graph.add((BLDG['{equipment_name1}'], BRICK.{BRICK_relation}, BLDG['{equipment_name2}']))"
    exec(dummy_code)

In [50]:
"""

# templates
oa_ra_damper_template = brick.get_template_by_name(BRICK.Damper)
fan_template = brick.get_template_by_name(BRICK.Supply_Fan)
clg_coil_template = brick.get_template_by_name(BRICK.Cooling_Coil)

# add fan
fan_name = f"{ahu_name}-Fan"
fan_binding_dict = {"name": BLDG[fan_name]}
fan_graph = fan_template.evaluate(fan_binding_dict)
model.add_graph(fan_graph)

# add outdoor air/return air damper
oa_ra_damper_name = f"{ahu_name}-Damper"
oa_ra_damper_binding_dict = {"name": BLDG[oa_ra_damper_name]}
oa_ra_damper_graph = oa_ra_damper_template.evaluate(oa_ra_damper_binding_dict)
model.add_graph(oa_ra_damper_graph)

# add clg coil
clg_coil_name = f"{ahu_name}-Clg_Coil"
clg_coil_binding_dict = {"name": BLDG[clg_coil_name]}
clg_coil_graph = clg_coil_template.evaluate(clg_coil_binding_dict)
model.add_graph(clg_coil_graph)

# connect fan, damper, and clg coil to AHU
model.graph.add((BLDG[ahu_name], BRICK.hasPart, BLDG[oa_ra_damper_name]))
model.graph.add((BLDG[ahu_name], BRICK.hasPart, BLDG[fan_name]))
model.graph.add((BLDG[ahu_name], BRICK.hasPart, BLDG[clg_coil_name]))
"""
# print model to confirm components were added and connected
print(model.graph.serialize())


@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<test:building/> a owl:Ontology .

<test:building/Bldg> a brick:Building ;
    brick:hasPart <test:building/AdminSpace>,
        <test:building/Office> .

<test:building/FCU1> a brick:FCU ;
    brick:feeds <test:building/Zone1> .

<test:building/FCU2> a brick:FCU ;
    brick:feeds <test:building/Zone1> .

<test:building/FCU3> a brick:FCU ;
    brick:feeds <test:building/Zone1> .

<test:building/FCU4> a brick:FCU ;
    brick:feeds <test:building/Zone1> .

<test:building/FCU5> a brick:FCU ;
    brick:feeds <test:building/Zone1> .

<test:building/Furnace1> a brick:HVAC_Equipment ;
    brick:feeds <test:building/Zone1> ;
    brick:hasPart <test:building/Boiler1>,
        <test:building/Fan1>,
        <test:building/HeatEx1> .

<test:building/Furnace2> a brick:HVAC_Equipment ;
    brick:feeds <test:building/Zone1> ;
    brick:hasPart <test:building/Boiler2>,
        <test:building/Fan2>

In [51]:
#save model
model.graph.serialize(destination="sample_building_model.ttl")

<Graph identifier=7564883c-0911-49f1-a985-da954caf72b8 (<class 'rdflib.graph.Graph'>)>